Ce notebook utilise un modèle glove google préentrainé téléchargé ici: http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/. 
Notebook inspiré de https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout/notebook

In [41]:
import pandas as pd
import numpy as np
import configparser

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, GlobalMaxPool1D, Bidirectional, Embedding, Dropout
from keras.models import Model

# gensim
from gensim.models.keyedvectors import KeyedVectors

# sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 

# Chargement des données

In [2]:
config = configparser.ConfigParser()
config.read('../config.cfg')
EMBEDDING_FILE=config['FILES']['GOOGLE']
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [3]:
# Chargement des données sous forme de dataframes pandas
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

# Définition des différents labels disponibles sous forme de liste
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [4]:
embed_size = 300 # taille du vecteur
max_features = 20000 # nombre de mots uniques utilisés (i.e nombre de lignes dans le vecteur d'embedding)
maxlen = 100 # nombre maximum de mots à considérer dans un commentaire

# Tokenisation

In [5]:
# On garde les mots les plus fréquents dans le jeu d'entrainement pour établir notre liste de tokens.
tokenizer = Tokenizer(num_words=max_features)
# Calcul des mots les plus fréquents
tokenizer.fit_on_texts(list(list_sentences_train))
# Indexation des jeux d'entrainement et test, conversion du texte en séquence d'indexes
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

La documentation de Tokenizer: https://keras.io/preprocessing/text/
et la documentation sur ses fonctions ici : http://faroit.com/keras-docs/1.2.2/preprocessing/text/

In [6]:
# Les commentaires ont une longueur variable qui peut être inférieure à maxlen
# On complète donc la séquence avec des 0, par défaut au début de la séquence
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

La documentation de pad_sequence ici: https://keras.io/preprocessing/sequence/

In [7]:
# Chargement des vecteurs du modèle word2vec
# Le fichier est encodé en binaire (.bin) -> binary=True
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [8]:
word_index = tokenizer.word_index
# le modèle pré-entrainé contient 3 millions de mots contre 20000 pour notre jeu de données.
vocabulary_size=min(len(word_index)+1,max_features)

# la matrice d'embedding représente le vocabulaire final à utiliser. 
# On ne gardera que 20000 vecteurs du modèle pré-entrainé. Leur longueur est de 300.
embedding_matrix = np.zeros((vocabulary_size, embed_size))


for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector    
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

In [9]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Documentation d'input, dense et dropout: https://keras.io/layers/core/

Documentation d'embedding: https://keras.io/layers/embeddings/

Documentation des BRNNs: https://keras.io/layers/wrappers/

Documentation des RNN: https://keras.io/layers/recurrent/

Documentation du pooling: https://keras.io/layers/pooling/

Documentation de l'activation: https://keras.io/activations/

Documentation de l'optimiseur: https://keras.io/optimizers/

Documentation de la fonction de loss: https://keras.io/losses/

Documentation des metrics: https://keras.io/metrics/

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          6000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 100)          140400    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total para

In [11]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 774s 5ms/step - loss: 0.0566 - acc: 0.9804 - val_loss: 0.0466 - val_acc: 0.9827
Epoch 2/2
143613/143613 [==============================] - 755s 5ms/step - loss: 0.0404 - acc: 0.9846 - val_loss: 0.0451 - val_acc: 0.9835


In [12]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [13]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 55s 867us/step


[0.06546974438924606, 0.9729073932009209]

Ici, l'accuracy calculée est la moyenne des accuracys de chaque problème binaire. L'accuracy de chaque problème binaire est le ratio entre le nombre de réponses correctes et le nombre de commentaires à classifier (// à la fonction accuracy_score de sklearn).

In [15]:
y_pred_t = model.predict(X_te, batch_size=1024)

In [22]:
print(accuracy_score(yt, y_pred_t[test_label_strip.index].round()))

0.8853824752258589


Pour un problème multi-label, la fonction accuracy_score considère le groupe de label entier et pas des sous problèmes binaires. Par exemple, si on prédit [1,0,0,0,0,0] pour un commentaire alors qu'en réalité les labels sont [1,0,1,0,0,0], on considérera la réponse fausse (valeur 0) même si on avait réussi à prédire le premier label. On prédit plus rarement tout les labels correctements pour un même commentaire qu'on ne prédit indépendamment chaque label. 

In [46]:
# Classification pour chaque label de façon indépendante
list_acc = [] 
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt[:,label], y_pred_t[test_label_strip.index, label].round())
    list_acc.append(acc)
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt[:,label], y_pred_t[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt[:,label], y_pred_t[test_label_strip.index, label].round()) )
print('La Moyenne des accuracy est de {}'.format(np.mean(list_acc)))
          

... Traitement du label toxic
L'accuracy du jeu test est 0.9275844821657445
Matrice de confusion sur le jeu test:
[[54294  3594]
 [ 1039  5051]]
Rapport : 
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     57888
           1       0.58      0.83      0.69      6090

   micro avg       0.93      0.93      0.93     63978
   macro avg       0.78      0.88      0.82     63978
weighted avg       0.94      0.93      0.93     63978

... Traitement du label severe_toxic
L'accuracy du jeu test est 0.9935915470943136
Matrice de confusion sur le jeu test:
[[63504   107]
 [  303    64]]
Rapport : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.37      0.17      0.24       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.68      0.59      0.62     63978
weighted avg       0.99      0.99      0.99     63978

... Traitement du label obs

Si on calcule l'accuracy pour chaque label de façon indépendante, les labels peu représentés (threat) sont en réalité mal prédits. On reconfirme le calcul effectué par Keras.

In [67]:
y_tan = y
y_tan = np.where(y_tan==0,-1,y_tan)

On remplace les 0 par des -1 en vue d'appliquer la fonction d'activation tanh et la loss hinge. 

In [62]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="tanh")(x)
model2 = Model(inputs=inp, outputs=x)
model2.compile(loss='hinge', optimizer='adam', metrics=['accuracy'])

In [68]:
model2.fit(X_t, y_tan, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 768s 5ms/step - loss: 0.9635 - acc: 0.8329 - val_loss: 0.9627 - val_acc: 0.9620
Epoch 2/2
143613/143613 [==============================] - 869s 6ms/step - loss: 0.9634 - acc: 0.9427 - val_loss: 0.9627 - val_acc: 0.9940


In [69]:
yt_tan = yt
yt_tan = np.where(yt_tan==0,-1,yt_tan)

In [70]:
model2.evaluate(X_te[test_label_strip.index], yt_tan, batch_size=1024)

16384/63978 [======>.......................] - ETA: 45s

KeyboardInterrupt: 

In [ ]:
y_pred_t2 = model2.predict(X_te, batch_size=1024)

In [ ]:
print(accuracy_score(yt_tan, y_pred_t2[test_label_strip.index].round()))

In [ ]:
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt_tan[:,label], y_pred_t2[test_label_strip.index, label].round())
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt_tan[:,label], y_pred_t2[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt_tan[:,label], y_pred_t2[test_label_strip.index, label].round()) )